In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot
sns.set(style="darkgrid")
pd.options.display.max_columns = 999
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# DataFrame
df = pd.read_csv("BankChurners_v2.csv")

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
# Cleaning data
df.shape

In [ ]:
df["CLIENTNUM"].nunique()

In [7]:
# Duplicates
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [10]:
# Subsetting 
df = df[['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']]

In [ ]:
df.dtypes

In [ ]:
# Missing values
df.isnull().sum()

In [ ]:
# We can fill the missing values by using following code:
df["Education_Level"] = df["Education_Level"].fillna("Unknown")
# Verify
df[df["Education_Level"]=="Unknown"]

In [ ]:
# Binning
print(f"Minimum age: {df["Customer_Age"].min()}")
print(f"Maximum age: {df["Customer_Age"].max()}")

In [15]:
bins = [25, 30, 40, 50, 60, 70, 80]
labels = ["20s", "30s", "40s", "50s", "60s", "70s"]
df["Customer_Age_bins"] = pd.cut(df["Customer_Age"], bins=bins, labels=labels, include_lowest=True, right=False)

In [ ]:
df[df["Customer_Age"]==70]

In [ ]:
# EDA
# Sanity checks
df["Attrition_Flag"].value_counts()

In [ ]:
print(round(df["Attrition_Flag"].value_counts()["Attrited Customer"] / df.shape[0] * 100, 2), "% of our customers have churned.")

In [ ]:
# Summary statistics
df.describe()

In [ ]:
print(f"The average Credit_Limit is ${round(np.mean(df["Credit_Limit"]), 2)} and the median is ${round(np.median(df["Credit_Limit"]), 2)}")

In [ ]:
# Distributions
# Histogram for "Customer_Age"
plt.hist(df["Customer_Age"]);

In [ ]:
# Histogram for "Months_on_book"
plt.hist(df["Months_on_book"]);

In [ ]:
plt.hist(df["Credit_Limit"]);

In [ ]:
# To get have better understanding of the histogram for Credit_Limit
plt.figure(figsize=(10, 7))
plt.hist(df["Credit_Limit"], bins=30)

plt.vlines(df["Credit_Limit"].mean(), 0, 2500, colors="black")
plt.vlines(df["Credit_Limit"].median(), 0, 2500, colors="black")
plt.text(df["Credit_Limit"].mean()-1000, 2500+50, "Mean")
plt.text(df["Credit_Limit"].median()-1000, 2500+50, "Median")
plt.ylim(0, 2800)
plt.title("Histogram of Customer Credit Limit")
plt.ylabel("Frequency")
plt.xlabel("Credit Limit");

In [ ]:
plt.hist(df["Total_Trans_Ct"]);

In [ ]:
plt.hist(df["Total_Trans_Ct"], bins=50);

In [27]:
# Data Transformations: Normalization and Log
def normalize(column):
    upper = column.max()
    lower = column.min()
    y = (column - lower) / (upper - lower)
    return y

In [28]:
df["Credit_Limit_Normalized"] = normalize(df["Credit_Limit"])
df["Credit_Limit_Log_Transformed"] = np.log(df["Credit_Limit"])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15,10))
fig.suptitle("Before and After Transformation")

# Boxplot in each subplot
sns.histplot(df, x="Credit_Limit", ax=axes[0,0])
sns.histplot(df, x="Credit_Limit_Normalized", ax=axes[0,1])
sns.histplot(df, x="Credit_Limit", ax=axes[1,0])
sns.histplot(df, x="Credit_Limit_Log_Transformed", ax=axes[1,1]);

In [ ]:
# Other distributions plots
sns.boxplot(x=df["Gender"], y=df["Total_Trans_Ct"]);

In [31]:
pyramid = df.groupby(["Gender", "Customer_Age_bins"], observed=False)["CLIENTNUM"].nunique().reset_index()

In [ ]:
pyramid

In [ ]:
women_bins = np.array(-1 * pyramid[pyramid["Gender"] == "F"]["CLIENTNUM"])
men_bins = np.array(pyramid[pyramid["Gender"] == "M"]["CLIENTNUM"])

y = list(range(0, 100, 10))

layout = go.Layout(yaxis=go.layout.YAxis(title='Age'),
                   xaxis=go.layout.XAxis(
                       range=[-1200, 1200],
                       tickvals=[-1000, -700, -300, 0, 300, 700, 1000],
                       ticktext=[1000, 700, 300, 0, 300, 700, 1000],
                       title='Number'),
                   barmode='overlay',
                   bargap=0.1)

data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='Men',
               hoverinfo='x',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='Women',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='seagreen')
               )]

iplot(dict(data=data, layout=layout))

In [ ]:
# Comparing Categories
sns.barplot(x="Customer_Age_bins", y="Credit_Limit", hue="Gender", data=df, estimator=np.mean);

In [ ]:
sns.barplot(x="Customer_Age_bins", y="Credit_Limit", hue="Gender", data=df, estimator=np.median);

In [ ]:
barplot = df.groupby(["Customer_Age_bins", "Gender"])["Credit_Limit"].mean().reset_index()
barplot

In [ ]:
sns.barplot(x="Customer_Age_bins", y="Credit_Limit", hue="Gender", data=barplot, estimator=np.mean);

In [ ]:
sns.barplot(x="Credit_Limit", y="Customer_Age_bins", hue="Gender", data=barplot, estimator=np.mean, orient="h");

In [ ]:
lollipop = df.groupby(["Customer_Age_bins"])["Credit_Limit"].mean().reset_index().sort_values("Credit_Limit")
lollipop

In [ ]:
plt.stem(lollipop["Customer_Age_bins"], lollipop["Credit_Limit"], linefmt="--");

In [ ]:
# Data Tables
df.groupby(["Attrition_Flag"]).agg({
    "CLIENTNUM":"nunique",
    "Customer_Age":"mean",
    "Dependent_count":"mean",
    "Months_on_book":"mean",
    "Total_Relationship_Count":"mean",
    "Months_Inactive_12_mon":"mean",
    "Contacts_Count_12_mon":"mean",
    "Credit_Limit":"mean",
    "Total_Revolving_Bal":"mean",
    "Avg_Open_To_Buy":"mean",
    "Total_Amt_Chng_Q4_Q1":"mean",
    "Total_Trans_Amt":"mean",
    "Total_Trans_Ct":"mean",
    "Total_Ct_Chng_Q4_Q1":"mean",
    "Avg_Utilization_Ratio":"mean"
})

In [ ]:
# Transpose
pivot_table = df.groupby(["Attrition_Flag"]).agg({
    "CLIENTNUM":"nunique",
    "Customer_Age":"mean",
    "Dependent_count":"mean",
    "Months_on_book":"mean",
    "Total_Relationship_Count":"mean",
    "Months_Inactive_12_mon":"mean",
    "Contacts_Count_12_mon":"mean",
    "Credit_Limit":"mean",
    "Total_Revolving_Bal":"mean",
    "Avg_Open_To_Buy":"mean",
    "Total_Amt_Chng_Q4_Q1":"mean",
    "Total_Trans_Amt":"mean",
    "Total_Trans_Ct":"mean",
    "Total_Ct_Chng_Q4_Q1":"mean",
    "Avg_Utilization_Ratio":"mean"
}).T

pivot_table

In [ ]:
pivot_table["Diff"] = pivot_table["Attrited Customer"] / pivot_table["Existing Customer"] - 1
pivot_table.sort_values("Diff")

In [ ]:
# Relationships
# Pairplot or Pairgrid
numeric_data = df._get_numeric_data()
numeric_data.columns

In [45]:
numeric_data = df[['CLIENTNUM', 'Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Credit_Limit_Normalized', 'Credit_Limit_Log_Transformed']]

In [ ]:
g = sns.PairGrid(numeric_data, diag_sharey=False, corner=True)
g.map_lower(sns.scatterplot)
g.map_diag(sns.histplot)

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(x="Total_Trans_Amt", y="Total_Trans_Ct", data=df)

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(x="Total_Trans_Amt", y="Total_Trans_Ct", hue="Attrition_Flag", data=df)

In [49]:
# Findings from the chart:
# 1. Customer Groups:
#    - "Existing Customers" tend to make more transactions (Total_Trans_Ct) and spend more money (Total_Trans_Amt).
#    - "Attrited Customers" (those who left) make fewer transactions and spend less.
# 2. Spending and Transactions Relationship:
#    - For both groups, spending more money generally means making more transactions.
#    - However, "Existing Customers" show a wider range of spending and transaction amounts.
# 3. Differences Between Groups:
#    - "Attrited Customers" mostly fall into the range of 20-80 transactions and spend less than 10,000.
#    - "Existing Customers" are more spread out and often make more transactions and spend more.
# 4. Business Insights:
#    - The customers who left ("Attrited Customers") could be targeted with campaigns to bring them back.
#    - Special offers or loyalty programs could be used to retain "Existing Customers," especially those who spend a lot.


In [50]:
bins = [0, 11000, 900000]
labels = ["Group 1", "Group 2"]
df["Total_Trans_Amt_bin"] = pd.cut(df["Total_Trans_Amt"], bins=bins, labels=labels, include_lowest=True, right=False)

In [ ]:
df.groupby(["Total_Trans_Amt_bin", "Attrition_Flag"]).agg({
    "CLIENTNUM":"nunique",
    "Customer_Age":"mean",
    "Dependent_count":"mean",
    "Months_on_book":"mean",
    "Total_Relationship_Count":"mean",
    "Months_Inactive_12_mon":"mean",
    "Contacts_Count_12_mon":"mean",
    "Credit_Limit":"mean",
    "Total_Revolving_Bal":"mean",
    "Avg_Open_To_Buy":"mean",
    "Total_Amt_Chng_Q4_Q1":"mean",
    "Total_Trans_Amt":"mean",
    "Total_Trans_Ct":"mean",
    "Total_Ct_Chng_Q4_Q1":"mean",
    "Avg_Utilization_Ratio":"mean"
}).T

In [52]:
# Total_Revolving_Bal (still a good indicator)
# Avg_Utilization_Ratio (not good anymore, since attrited and existing have similar values)
# Total_Trans_Ct (still a good indicator)
# Total_Trans_AMT (not good anymore, since att. and exi. have similar values)

In [ ]:
# KDE plots
g = sns.FacetGrid(df, row="Attrition_Flag", aspect=5, height=3)
g.map_dataframe(sns.kdeplot, x="Total_Ct_Chng_Q4_Q1")

plt.xlim(0, 2)

In [54]:
# Changing the background color and removing the border
sns.set_theme(style="white")
palette = sns.color_palette("Set2", 12)

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(x="Total_Trans_Amt", y="Total_Trans_Ct", hue="Attrition_Flag", data=df)

sns.despine(bottom=True, left=True)
plt.ylim(0, 150)

plt.xlabel("Total Transaction Amount")
plt.ylabel("Total Transaction Count")
plt.title("Transaction Amounts and Counts for Customers", loc="left", size=14)

plt.vlines(11000, 0, 150)
plt.text(12500, 40, "No Churned Customers \nabove $11K of Spend")

In [ ]:
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0), "axes.linewidth":2})
palette = sns.color_palette("Set2", 12)
g = sns.FacetGrid(df, palette=palette, row="Attrition_Flag", hue="Attrition_Flag", aspect=5, height=3)
g.map_dataframe(sns.kdeplot, x="Total_Ct_Chng_Q4_Q1", fill=True, alpha=1)
g.map_dataframe(sns.kdeplot, x="Total_Ct_Chng_Q4_Q1", color="black")

def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, color="black", fontsize=13, ha="left", va="center", transform=ax.transAxes)

g.map(label, "Attrition_Flag")
g.fig.subplots_adjust(hspace=-.5)
g.set_titles("")
g.set(yticks=[], xlabel="Total Count Change Q4 to Q1")
g.despine(left=True)
plt.suptitle("Customer Q4 to Q1 Transaction Drop", y=0.98)

plt.xlim(0,2);